# Simpsons characters dialogs

In [12]:
import pandas as pd
from flask import Flask, request

In [13]:
df = pd.read_csv('input/simpsons_dataset.csv', engine = 'python')
df.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [14]:
nombres = { 
    'raw_character_text':'Characters',
    'spoken_works':'Text'
}
df = df.rename(columns=nombres)

# Create characters groups

In [15]:
df = df = df.dropna()
df = df[(df['Characters'].isin(('Homer Simpson', 'Marge Simpson', 'Bart Simpson', 'Lisa Simpson', 'Maggie Simpson', 'Abraham Simpson', 'Milhouse Van Houten', 'Ralph Wiggum', 'Nelson Muntz', 'Seymour Skinner', 'Edna Krabappel', 'Groundskeeper Willie', 'Jimbo Jones', 'Martin Prince', 'Moe Szyslak', 'Barney Gumble', 'Lenny Leonard', 'Carl Carlson')))]
df.head()

,Characters,spoken_words
1,Lisa Simpson,Where's Mr. Bergstrom?
3,Lisa Simpson,That life is worth living.
5,Martin Prince,I don't think there's anything left to say.
7,Bart Simpson,Victory party under the slide!
9,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!


In [16]:
df['Groups'] = df['Groups'][df['Characters'].isin(('Homer Simpson', 'Marge Simpson', 'Bart Simpson', 'Lisa Simpson', 'Maggie Simpson', 'Abraham Simpson'))] = 'Home'
df['Groups'][df['Characters'].isin(('Milhouse Van Houten', 'Ralph Wiggum', 'Nelson Muntz', 'Seymour Skinner', 'Edna Krabappel', 'Groundskeeper Willie', 'Jimbo Jones', 'Martin Prince'))] = 'School'
df['Groups'][df['Characters'].isin(('Moe Szyslak', 'Barney Gumble', 'Lenny Leonard', 'Carl Carlson'))] = 'Bar'

In [17]:
df_reduced = df.head(100)
df_reduced.head()

,Characters,spoken_words,Groups
1,Lisa Simpson,Where's Mr. Bergstrom?,Home
3,Lisa Simpson,That life is worth living.,Home
5,Martin Prince,I don't think there's anything left to say.,School
7,Bart Simpson,Victory party under the slide!,Home
9,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!,Home


# Including Simpsons characters on MongoDB

In [18]:
import requests

In [ ]:
ch = df[['Characters', 'Groups']].drop_duplicates()
ch

In [ ]:
for character in pd.unique(df_reduced['Characters']):
    data = requests.get(f"http://localhost:8800/user/create/{character}").content
print(data)

# Creating chats

In [ ]:
for group in pd.unique(df_reduced['Groups']):
    data = requests.get(f"http://localhost:8800/chat/create/{group}").content
print(data)

# Adding characters to chat

In [ ]:
for index, row in ch.iterrows():
    print(row['Characters'], row['Groups'])
    users = requests.get(f"http://localhost:8800/user/create/{row['Characters']}/{row['Groups']}").content
    print(users)

In [ ]:

for character in pd.unique(df_reduced['Characters']):
    print(character)
    usersdata = requests.get(f"http://localhost:8800/chat/adduser/{character}").text
    print(usersdata)

# Adding messages to chat

In [ ]:
'''
Characters from the same group will be on the same MongoDB document conversation
'''
for index, row in df_reduced.iterrows():  
    data = requests.get(f"http://localhost:8800/chat/{row['Groups']}/{row['Characters']}/{row['spoken_words']}").content
print(data)

# Getting messages from chat

In [ ]:
#If you wanna have username + message:
groups = db['Conversations'].distinct('Group')

for group in groups:
    conversations = requests.get(f"http://localhost:8800/chat/{group}/list").text

    print(conversations)



In [ ]:
#If you wanna have a list with all messages:
groups = db['Conversations'].distinct('Group')

for group in groups:
    messages = requests.get(f"http://localhost:8800/chat/{group}/userlist").text

    print(messages)

# Sentiment analysis

# Pruebas

In [40]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson.json_util import dumps
import ast

In [21]:
client = MongoClient("mongodb://localhost/Conversations")
db = client.get_database()


In [22]:
import nltk

In [23]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/TryAgain/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [24]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [25]:
sia = SentimentIntensityAnalyzer()

In [47]:
messages = requests.get(f"http://localhost:8800/chat/Home/userlist").text

a = ast.literal_eval(messages)
print(type(messages))
print(type(a))

<class 'str'>
<class 'list'>


In [65]:
print(a)

["Where's Mr. Bergstrom", 'That life is worth living.', 'Victory party under the slide!', 'Mr. Bergstrom! Mr. Bergstrom!', 'Do you know where I could find him', 'The train, how like him... traditional, yet environmentally sound.', 'I see he touched you, too.', 'Hey, thanks for your vote, man.', 'Well, you got that right. Thanks for your vote, girls.', "Well, don't sweat it. Just so long as a couple of people did... right, Milhouse", 'Lewis', 'Somebody must have voted.', 'Uh oh.', 'I demand a recount.', 'No.', 'Mr. Bergstrom! Hey, Mr. Bergstrom!', 'Hey, Lisa, indeed.', 'Oh, I mean, were you just going to leave, just like that', "You can't go! You're the best teacher I'll ever have.", 'Oh, please.', 'But I need you too.', "I, I understand. Mr. Bergstrom, I'm going to miss you.", 'Thank you, Mr. Bergstrom.', 'So, I guess this is it', 'Never thrown a party', "Bart didn't get one vote", 'Nothing.', 'Lisa, tell your father.', 'Mr. Bergstrom left today.', 'Oh.', "He's gone. Forever.", 'And', 

In [48]:
#Para cada frase
for phrase in a:
    b = sia.polarity_scores(phrase)
    print(b)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.678, 'pos': 0.322, 'compound': 0.2263}
{'neg': 0.0, 'neu': 0.572, 'pos': 0.428, 'compound': 0.4574}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.762, 'pos': 0.238, 'compound': 0.3612}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.633, 'pos': 0.367, 'compound': 0.4404}
{'neg': 0.0, 'neu': 0.615, 'pos': 0.385, 'compound': 0.6124}
{'neg': 0.0, 'neu': 0.861, 'pos': 0.139, 'compound': 0.2732}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.6, 'neu': 0.4, 'pos': 0.0, 'compound': -0.128}
{'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound': -0.296}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.094, 'neu': 0.709, 'pos'

In [68]:
#list to string
texto = ''.join([str(phrase) for phrase in messages])
texto

'["Where\'s Mr. Bergstrom", "That life is worth living.", "Victory party under the slide!", "Mr. Bergstrom! Mr. Bergstrom!", "Do you know where I could find him", "The train, how like him... traditional, yet environmentally sound.", "I see he touched you, too.", "Hey, thanks for your vote, man.", "Well, you got that right. Thanks for your vote, girls.", "Well, don\'t sweat it. Just so long as a couple of people did... right, Milhouse", "Lewis", "Somebody must have voted.", "Uh oh.", "I demand a recount.", "No.", "Mr. Bergstrom! Hey, Mr. Bergstrom!", "Hey, Lisa, indeed.", "Oh, I mean, were you just going to leave, just like that", "You can\'t go! You\'re the best teacher I\'ll ever have.", "Oh, please.", "But I need you too.", "I, I understand. Mr. Bergstrom, I\'m going to miss you.", "Thank you, Mr. Bergstrom.", "So, I guess this is it", "Never thrown a party", "Bart didn\'t get one vote", "Nothing.", "Lisa, tell your father.", "Mr. Bergstrom left today.", "Oh.", "He\'s gone. Forever."

In [59]:
#En conjunto

sia.polarity_scores(texto)

{'neg': 0.052, 'neu': 0.836, 'pos': 0.112, 'compound': 0.9927}

In [71]:
messages

'["Where\'s Mr. Bergstrom", "That life is worth living.", "Victory party under the slide!", "Mr. Bergstrom! Mr. Bergstrom!", "Do you know where I could find him", "The train, how like him... traditional, yet environmentally sound.", "I see he touched you, too.", "Hey, thanks for your vote, man.", "Well, you got that right. Thanks for your vote, girls.", "Well, don\'t sweat it. Just so long as a couple of people did... right, Milhouse", "Lewis", "Somebody must have voted.", "Uh oh.", "I demand a recount.", "No.", "Mr. Bergstrom! Hey, Mr. Bergstrom!", "Hey, Lisa, indeed.", "Oh, I mean, were you just going to leave, just like that", "You can\'t go! You\'re the best teacher I\'ll ever have.", "Oh, please.", "But I need you too.", "I, I understand. Mr. Bergstrom, I\'m going to miss you.", "Thank you, Mr. Bergstrom.", "So, I guess this is it", "Never thrown a party", "Bart didn\'t get one vote", "Nothing.", "Lisa, tell your father.", "Mr. Bergstrom left today.", "Oh.", "He\'s gone. Forever."

In [72]:
'''
Estructura:

sia.polarity_scores(variable)
La variable necesita ser string

Cuando es un string de una lista de frases, primero convertir a lista:
a = ast.literal_eval(messages)

Después 

'''
print(a)

["Where's Mr. Bergstrom", 'That life is worth living.', 'Victory party under the slide!', 'Mr. Bergstrom! Mr. Bergstrom!', 'Do you know where I could find him', 'The train, how like him... traditional, yet environmentally sound.', 'I see he touched you, too.', 'Hey, thanks for your vote, man.', 'Well, you got that right. Thanks for your vote, girls.', "Well, don't sweat it. Just so long as a couple of people did... right, Milhouse", 'Lewis', 'Somebody must have voted.', 'Uh oh.', 'I demand a recount.', 'No.', 'Mr. Bergstrom! Hey, Mr. Bergstrom!', 'Hey, Lisa, indeed.', 'Oh, I mean, were you just going to leave, just like that', "You can't go! You're the best teacher I'll ever have.", 'Oh, please.', 'But I need you too.', "I, I understand. Mr. Bergstrom, I'm going to miss you.", 'Thank you, Mr. Bergstrom.', 'So, I guess this is it', 'Never thrown a party', "Bart didn't get one vote", 'Nothing.', 'Lisa, tell your father.', 'Mr. Bergstrom left today.', 'Oh.', "He's gone. Forever.", 'And', 

In [74]:
sia.polarity_scores(messages)

{'neg': 0.052, 'neu': 0.836, 'pos': 0.112, 'compound': 0.9927}